<a href="https://colab.research.google.com/github/Armaanml/-Armaan-technical-/blob/main/web_crawling_mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import re

In [5]:
!pip install requests
!pip install requests-html
!pip install beautifulsoup4


In [8]:
!pip install lxml_html_clean


In [9]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urlsplit
import pandas as pd
from requests_html import HTMLSession
import threading
import time
from tqdm import tqdm

In [10]:
#user_url = "https://www.thapar.edu/sitemap.xml"
#url = ["https://www.thapar.edu"]
EMAIL_REGEX = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

In [11]:
def get_urls_of_xml(xml_url):
    r = requests.get(xml_url)
    xml = r.text
    soup = BeautifulSoup(xml)

    links_arr = []
    for link in soup.findAll('loc'):
        linkstr = link.getText('', True)
        links_arr.append(linkstr)

    return links_arr



links_data_arr = get_urls_of_xml("https://thapar.edu/sitemap.xml")
#print(links_data_arr)


/tmp/ipython-input-11-2655839664.py:4: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(xml)
/tmp/ipython-input-11-2655839664.py:7: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for link in soup.findAll('loc'):


In [12]:
session = HTMLSession()

In [15]:
EMAIL_REGEX = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'


In [16]:
email = set()
for i in tqdm(links_data_arr):
    try:
        r = session.get(i, timeout=10)
        raw_html = r.html.raw_html.decode(errors='ignore')
        for re_match in re.finditer(EMAIL_REGEX, raw_html):
            email.add(re_match.group().replace("-", ""))
    except Exception as e:
        print(f"Error processing {i}: {e}")


 37%|███▋      | 1615/4342 [23:10<35:35,  1.28it/s]

Error processing http://lmtsom.thapar.edu: HTTPConnectionPool(host='lmtsom.thapar.edu', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79cc7b390cd0>: Failed to establish a new connection: [Errno -2] Name or service not known'))


 38%|███▊      | 1651/4342 [23:39<27:17,  1.64it/s]

Error processing http://admissions.thapar.edu/?_ga=2.45427623.2046722684.1520064571-176534119.1510131791: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


 72%|███████▏  | 3141/4342 [45:32<40:04,  2.00s/it]

Error processing http://som.thapar.edu/misces/latestpages/email-account-support-during-covid-19: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|██████████| 4342/4342 [1:05:53<00:00,  1.10it/s]


In [17]:
df = pd.DataFrame(email)
df.to_csv('Emails.csv', index=False)